In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

1. Eliminar "Second Experiment" desde la UI.
2. En la terminal: `# rm -rf mlruns/.trash/*`

In [ ]:
# Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

classes, counts = np.unique(y, return_counts=True)
total = counts.sum()
percentages = (counts / total) * 100

# Imprimir resultados
print(f"Clase 0: {percentages[0]:.1f}% ({counts[0]} muestras)")
print(f"Clase 1: {percentages[1]:.1f}% ({counts[1]} muestras)")

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

Vamos a entrenar 4 modelos diferentes:
1. Regresión Logística
2. Random Forest
3. XGBoost
4. XGBoost con SMOTE

In [ ]:
smt = SMOTE(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

classes, counts = np.unique(y_train_res, return_counts=True)
total = counts.sum()
percentages = (counts / total) * 100

# Imprimir resultados
print(f"Clase 0: {percentages[0]:.1f}% ({counts[0]} muestras)")
print(f"Clase 1: {percentages[1]:.1f}% ({counts[1]} muestras)")

In [ ]:
models = [
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear'), 
        (X_train, y_train),
        (X_test, y_test),
        {"C": 1, "solver": "liblinear"}
    ),
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test),
        {"n_estimators": 30, "max_depth": 3}
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test),
        {"use_label_encoder": False, "eval_metric": "logloss"}
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_res, y_train_res),
        (X_test, y_test),
        {"use_label_encoder": False, "eval_metric": "logloss", "resampling": "SMOTE"}
    )
]

In [ ]:
reports = []

for model_name, model, train_set, test_set, params in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    print(model_name, "\n", report)
    reports.append(report)

➡️ Vamos a registrar las métricas y modelos en MLflow

In [ ]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from mlflow.models.signature import infer_signature

In [ ]:
params

In [ ]:
mlflow.set_experiment("Second Experiment")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    X_train, y_train = element[2]
    X_test, y_test = element[3]
    params = element[4]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):   
        # Log de hiperparámetros y nombre del modelo     
        mlflow.log_param("model", model_name)
        mlflow.log_params(params)
        
        # Log de todas las métricas
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                for metric_name, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric_name}", value)
            else:
                mlflow.log_metric(label, metrics)
                
        # Log del dataset
        X_train_df = pd.DataFrame(X_train, columns=[f"feature_{i}" for i in range(X_train.shape[1])])
        X_train_df["label"] = y_train
        dataset_path = f"data/train_dataset_{model_name.replace(' ', '_')}.csv"
        X_train_df.to_csv(dataset_path, index=False)
        mlflow.log_artifact(dataset_path, artifact_path="datasets")
        mlflow.set_tag("dataset", "with_smote") if "SMOTE" in model_name else mlflow.set_tag("dataset", "original")

        # Log del modelo
        input_example = pd.DataFrame(X_test[:2], columns=[f"feature_{i}" for i in range(X.shape[1])])    
        signature = infer_signature(X_test, y_pred)
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, 
                                     "model", 
                                     input_example = input_example, 
                                     signature = signature)
        else:
            mlflow.sklearn.log_model(model, 
                                     "model",
                                     input_example = input_example, 
                                     signature = signature)  

### 1. Registrar el Modelo

In [ ]:
model_name = 'XGB-Smote' # Unique way to call our model
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

💡 Si corremos la celda anterior nuevamente, vamos a estar cambiando de versión al modelo.

#### 🏷️ Aliases comunes en el Model Registry

Cuando registramos modelos en el **Model Registry** de MLflow, podemos asignarles **etiquetas (aliases)** para indicar su rol dentro del ciclo de vida del proyecto.

Algunos alias comunes:

- **Champion**: es el **modelo actual en producción**, el mejor hasta el momento.
- **Challenger**: es un **modelo nuevo que compite** con el Champion. Se lo entrena y evalúa para ver si lo supera.
- **Staging**: modelo en etapa de pruebas, listo para ser validado antes de pasarlo a producción.
- **Archived**: modelos viejos que ya no se usan, pero que quedan guardados para referencia o auditoría.

💡 Usar estos nombres no es obligatorio, pero es una **convención muy útil** para que todo el equipo entienda rápidamente el rol de cada modelo registrado.

### 2. Generar predicciones con el modelo *Challenger*

Ahora vamos a cargar el modelo que registramos como **Challenger** desde el **Model Registry** de MLflow y vamos a **testearlo localmente**.

Esto nos permite, por ejemplo:
- Validar su performance con datos nuevos
- Compararlo contra el modelo actual en producción (el *Champion*)
- Decidir si vale la pena promoverlo a producción


In [ ]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

Otra opcion usando el alias en lugar del model_version:

In [ ]:
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

### 3. Transicionar el modelo a Producción

Usamos MLflowClient

In [ ]:
dev_model_uri = f"models:/{model_name}@challenger"
production_model_name = "anomaly-detection-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri, dst_name=production_model_name)

➡️ Ahora le podemos dar un alias en la UI, como `"champion"`


Una vez que ya tenemos identificado y probado nuestro modelo Champion, podríamos por ejemplo, empaquetarlo dentro de un contenedor Docker.

Además, hay muchas plataformas que **integran nativamente con MLflow**, como por ejemplo:

- **Databricks**
- **AWS SageMaker**
- **Azure ML**
- **Google Cloud Vertex AI**
- Y otras plataformas open source que aceptan modelos MLflow.

Esto nos permite tener un flujo de trabajo completo: desde la experimentación hasta el despliegue, todo trazable y reproducible.


In [ ]:
# ➡️ Una vez que ya le di el alias @champion, genero predicciones con el modelo en producción 
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]